In [1]:
import numpy as np
import pandas as pd
from pandas import to_datetime
from Air_statistics import Statistics
import queue
from collections import defaultdict

D14 = pd.read_csv('2014.csv')
D15 = pd.read_csv('2015.csv')
D16 = pd.read_csv('2016.csv')
D17 = pd.read_csv('2017.csv')
D18 = pd.read_csv('2018.csv')
D19 = pd.read_csv('2019.csv')

In [2]:
D = pd.concat([D14,D15,D16,D17,D18,D19])

In [3]:
D['Start_time'] = to_datetime(D['Start_time'],format="%Y-%m-%d %H:%M:%S")

In [4]:
D['Start_time']

0       2014-01-01 00:00:00
1       2014-01-01 00:00:00
2       2014-01-01 00:00:00
3       2014-01-01 00:00:00
4       2014-01-01 00:00:00
                ...        
78544   2019-12-31 23:00:00
78545   2019-12-31 23:00:00
78546   2019-12-31 23:00:00
78547   2019-12-31 23:00:00
78548   2019-12-31 23:00:00
Name: Start_time, Length: 399641, dtype: datetime64[ns]

In [5]:
D['month'] = pd.DatetimeIndex(D['Start_time']).month

In [6]:
D['year'] = pd.DatetimeIndex(D['Start_time']).year
D

,Unnamed: 0,Source,Target,Probability,Start_time,End_time,Trans_Volume,start_pm,end_pm,month,year
0,0,上海市,南通市,0.200,2014-01-01 00:00:00,2014-01-01 05:00:00,31.1800,155.9,152.8,1,2014
1,1,南京市,滁州市,0.400,2014-01-01 00:00:00,2014-01-01 08:00:00,60.0400,150.1,NaN,1,2014
2,2,南通市,泰州市,0.800,2014-01-01 00:00:00,2014-01-01 04:00:00,127.8400,159.8,118.3,1,2014
3,3,嘉兴市,苏州市,0.600,2014-01-01 00:00:00,2014-01-01 06:00:00,77.8200,129.7,96.8,1,2014
4,4,常州市,镇江市,0.800,2014-01-01 00:00:00,2014-01-01 06:00:00,106.8000,133.5,110.0,1,2014
...,...,...,...,...,...,...,...,...,...,...,...
78544,78544,湖州市,苏州市,0.125,2019-12-31 23:00:00,2020-01-01 03:00:00,3.9375,31.5,NaN,12,2019
78545,78545,芜湖市,马鞍山市,0.125,2019-12-31 23:00:00,2020-01-01 00:00:00,4.3750,35.0,NaN,12,2019
78546,78546,镇江市,常州市,0.250,2019-12-31 23:00:00,2020-01-01 07:00:00,8.5000,34.0,NaN,12,2019
78547,78547,阜阳市,淮南市,0.750,2019-12-31 23:00:00,2020-01-01 06:00:00,47.7750,63.7,NaN,12,2019


In [7]:
D_winter = D[D['month'].isin([1,2,12])]
s = set(D_winter['Source'].values)
t = set(D_winter['Target'].values)
city = s|t
dict_node_isvisited = {i:0 for i in city}

In [8]:
year = 2019
D_winter = D[D['month'].isin([1,2,12]) & (D['year'] == year)]
table = D_winter.groupby(["Source","Target"], as_index=False)['Trans_Volume'].sum()
def bfs(D,to_name,D_winter):
    dict_node_isvisited = {i:0 for i in city}
    q = queue.Queue()
    dict_node_isvisited[to_name] = 1
    res = defaultdict(list)
    q.put([to_name,0])
    while not q.empty():
        to_list = q.get()
        if len(D[D['Target']==to_list[0]]):
            for frm in D[D['Target']==to_list[0]]['Source'].values:
                if dict_node_isvisited[frm] == 0:
                    dict_node_isvisited[frm] = 1
                    summery = D_winter[(D_winter['Source']==frm) & (D_winter['Target']==to_list[0])].groupby(
                        ['Source','Target'],as_index=False)['Trans_Volume'].agg(['mean','count','std','sum'])
                    arr = [frm, to_list[0]]
                    arr.extend(list(summery.iloc[-1].values))
                    res[to_list[1]+1].append(arr)
                    q.put([frm,to_list[1]+1])
    return res
result = bfs(table,'上海市',D_winter)

#创建一个空的Dataframe
result_table =pd.DataFrame(columns=['Near_Class','Source','Target','mean','count','std','sum'])
for k in result.keys():
    for instance in result[k]:
        result_table=result_table.append(pd.DataFrame({'Near_Class':[k],
                                           'Source':[instance[0]],
                                           'Target':[instance[1]],
                                           'mean':[instance[2]],
                                           'count':[instance[3]],
                                           'std':[instance[4]],
                                           'sum':[instance[5]]}),ignore_index=True)
result_table.to_csv(str(year)+'_trans_patterns.csv')

In [22]:
from collections import defaultdict
dict_nn = defaultdict(list)
for _,r in result_table.iterrows():
    dict_nn[r[0]].append((r[1],r[2]))

In [49]:
from pyecharts import GeoLines, Style
def plot_bfs(dict_nn):
    outputname = 'all'
    D_winter = D[D['month'].isin([1,2,12])]# & (D['year'] == year)]
    table = D_winter.groupby(["Source","Target"], as_index=False)['Trans_Volume'].sum()
    result = bfs(table,'上海市',D_winter)

    #创建一个空的Dataframe
    result_table =pd.DataFrame(columns=['Near_Class','Source','Target','mean','count','std','sum'])
    for k in result.keys():
        for instance in result[k]:
            result_table=result_table.append(pd.DataFrame({'Near_Class':[k],
                                               'Source':[instance[0]],
                                               'Target':[instance[1]],
                                               'mean':[instance[2]],
                                               'count':[instance[3]],
                                               'std':[instance[4]],
                                               'sum':[instance[5]]}),ignore_index=True)
    result_table.to_csv(str(year)+'_trans_patterns.csv')
    dict_nn = defaultdict(list)
    for _,r in result_table.iterrows():
        dict_nn[r[0]].append((r[1],r[2]))
    style = Style(
        title_color="#fff",
        title_pos="center",
        width=1510,
        height=840,
        background_color="#404a59"  # #404a59"
    )

    style_geo = style.add(
        maptype="china",
        is_label_show=False,
        line_curve=0.1,
        line_opacity=0.6,
        legend_text_color="#eee",
        legend_pos="right",
        geo_effect_symbol="arrow",
        symbol_size=5,
        geo_effect_symbolsize=5,
        label_color=['#a6c84c', '#46bee9', 'red'],  # '#ffa022'],#'#a6c84c'
        label_pos="right",
        label_formatter="{b}",
        label_text_color="#eee",
        # geo_cities_coords=airports_geo
    )

    geolines = GeoLines(outputname, **style.init_style)
    for i in dict_nn.keys():
        geolines.add(name=str(i)+"阶", data=dict_nn[i],
                     is_legend_show=True, **style_geo)
    geolines.render(str(outputname)+'近邻城市.html')
    list_frame = []
    for k in dict_nn.keys():
        for frm,end in dict_nn[k]:
            list_frame.append(D_winter[(D_winter['Source']==frm) & (D_winter['Target']==end)])
    out = pd.concat(list_frame)
    out.to_csv(str(outputname)+'data_detail.csv')

In [50]:
plot_bfs(dict_nn)

In [27]:
dict_nn

defaultdict(list,
            {1: [('南通市', '上海市'),
              ('嘉兴市', '上海市'),
              ('无锡市', '上海市'),
              ('泰州市', '上海市'),
              ('盐城市', '上海市'),
              ('苏州市', '上海市')],
             2: [('常州市', '南通市'),
              ('镇江市', '南通市'),
              ('湖州市', '嘉兴市'),
              ('绍兴市', '嘉兴市'),
              ('舟山市', '嘉兴市'),
              ('宿迁市', '无锡市'),
              ('扬州市', '无锡市'),
              ('淮安市', '无锡市'),
              ('连云港市', '无锡市'),
              ('杭州市', '苏州市')],
             3: [('南京市', '常州市'),
              ('芜湖市', '常州市'),
              ('马鞍山市', '常州市'),
              ('滁州市', '镇江市'),
              ('宣城市', '湖州市'),
              ('宁波市', '绍兴市'),
              ('徐州市', '宿迁市')],
             4: [('合肥市', '南京市'),
              ('宿州市', '南京市'),
              ('蚌埠市', '南京市'),
              ('铜陵市', '芜湖市'),
              ('淮南市', '滁州市'),
              ('淮北市', '徐州市')],
             5: [('六安市', '合肥市'),
              ('安庆市', '合肥市'),
              ('阜阳市', '合肥市'),
 

In [30]:
list_frame = []
for k in dict_nn.keys():
    for frm,end in dict_nn[k]:
        list_frame.append(D_winter[(D_winter['Source']==frm) & (D_winter['Target']==end)])

In [32]:
out = pd.concat(list_frame)

In [33]:
out

,Unnamed: 0,Source,Target,Probability,Start_time,End_time,Trans_Volume,start_pm,end_pm,month,year
209,209,南通市,上海市,0.200,2019-01-01 23:00:00,2019-01-02 09:00:00,14.500,72.5,76.7,1,2019
229,229,南通市,上海市,0.500,2019-01-02 00:00:00,2019-01-02 10:00:00,36.000,72.0,82.0,1,2019
249,249,南通市,上海市,0.400,2019-01-02 01:00:00,2019-01-02 10:00:00,28.600,71.5,82.0,1,2019
284,284,南通市,上海市,0.200,2019-01-02 03:00:00,2019-01-02 11:00:00,14.700,73.5,79.1,1,2019
303,303,南通市,上海市,0.300,2019-01-02 04:00:00,2019-01-02 12:00:00,21.660,72.2,75.9,1,2019
...,...,...,...,...,...,...,...,...,...,...,...
77824,77824,池州市,安庆市,0.250,2019-12-29 12:00:00,2019-12-29 13:00:00,11.750,47.0,61.3,12,2019
77827,77827,池州市,安庆市,0.125,2019-12-29 15:00:00,2019-12-29 16:00:00,6.000,48.0,58.7,12,2019
77830,77830,池州市,安庆市,0.125,2019-12-29 17:00:00,2019-12-29 18:00:00,5.875,47.0,62.3,12,2019
77832,77832,池州市,安庆市,0.125,2019-12-29 19:00:00,2019-12-29 20:00:00,7.250,58.0,69.0,12,2019
